In [1]:
using InteractiveUtils, Test

In [2]:
function pw_rec(n, x)
    if iszero(n)
        one(x)
    else
        x * pw_rec(n - 1, x)
    end
end

pw_rec (generic function with 1 method)

In [3]:
pw_rec(3, "Abc")

"AbcAbcAbc"

In [4]:
Base.iszero(::Val{N}) where {N} = iszero(N)

In [5]:
Base.:-(::Val{n}, m) where {n} = Val{n - m}()

In [6]:
pw_rec(Val{3}(), 10)

1000

In [7]:
@code_typed pw_rec(Val{3}(), 10)

CodeInfo(
1 ─      nothing::Nothing
│   %2 = Base.mul_int(x, 1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
│   %4 = Base.mul_int(x, %3)::Int64
└──      return %4
) => Int64

In [8]:
abstract type MAbs{T} end

In [9]:
struct MOne{T} <: MAbs{T} end

In [10]:
struct MVal{T} <: MAbs{T}
    v::T
end

In [11]:
from_mabs(::MOne{T}) where {T} = one(T)
from_mabs(x::MVal{T}) where {T} = x.v

from_mabs (generic function with 2 methods)

In [12]:
from_mabs(MOne{Int}()), from_mabs(MVal("Abc"))

(1, "Abc")

In [13]:
to_mabs(x::T) where {T} =
    isone(x) ? MOne{T}() : MVal{T}(x)

to_mabs (generic function with 1 method)

In [14]:
to_mabs(""), to_mabs("Abc")

(MOne{String}(), MVal{String}("Abc"))

In [15]:
function pw_abs(n, x::MAbs{T}) where {T}
    if iszero(n)
        MOne{T}()
    else
        x * pw_abs(n - 1, x)
    end
end

pw_abs (generic function with 1 method)

In [16]:
Base.:*(::MOne{T}, y::MAbs{T}) where {T} = y

In [17]:
Base.:*(x::MVal{T}, y::MOne{T}) where {T} = x

In [18]:
Base.:*(x::MVal{T}, y::MVal{T}) where {T} = MVal{T}(x.v * y.v)

In [19]:
to_mabs("") * to_mabs(""), to_mabs("") * to_mabs("Abc"), to_mabs("Abc") * to_mabs("")

(MOne{String}(), MVal{String}("Abc"), MVal{String}("Abc"))

In [20]:
to_mabs("Abc") * to_mabs("Def")

MVal{String}("AbcDef")

In [21]:
pw_abs(Val(0), to_mabs("Abc"))

MOne{String}()

In [22]:
pw_abs(Val(1), to_mabs("Abc"))

MVal{String}("Abc")

In [23]:
pw_abs(Val(3), to_mabs("Abc"))

MVal{String}("AbcAbcAbc")

In [24]:
@code_typed pw_abs(Val(3), to_mabs(10))

CodeInfo(
1 ─      nothing::Nothing
│   %2 = Base.getfield(x, :v)::Int64
│   %3 = Base.getfield(x, :v)::Int64
│   %4 = Base.mul_int(%2, %3)::Int64
│   %5 = Base.getfield(x, :v)::Int64
│   %6 = Base.mul_int(%5, %4)::Int64
│   %7 = %new(MVal{Int64}, %6)::MVal{Int64}
└──      return %7
) => MVal{Int64}